## 导入数据

In [1]:
import numpy as np
import os
import pandas as pd
import pickle
import pandas as pd
import numpy as np
from scipy.interpolate import Rbf
from scipy.interpolate import interpn
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import math
from math import radians, cos, sin, asin, sqrt
import scipy.io as sio
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

In [2]:
## 读取WRF数据
with open('../data/wrf_dataset_init', 'rb') as f:
    wrf_init = pickle.load(f)

In [3]:
wrf_init.keys()

dict_keys(['time', 'u10', 'v10', 'rh2', 't2', 'slp', 'lon', 'lat', 'velocity', 'degree'])

In [4]:
## 读取Lable数据
with open('../data/real_dataset_noabnormal', 'rb') as f:
    real_target = pickle.load(f)

In [5]:
real_target.keys()

Index(['站位编号', '获取时间', '经度', '纬度', '平均风速', '平均风向', '平均风速_10m'], dtype='object')

## 构造WRF数据点

In [6]:
# WRF数据场
wrf_lon = wrf_init['lon']
wrf_lat = wrf_init['lat']
wrf_time = wrf_init['time']

wrf_velocity = wrf_init['velocity']
wrf_degree = wrf_init['degree']
wrf_u10 = wrf_init['u10']
wrf_v10 = wrf_init['v10']
wrf_rh2 = wrf_init['rh2']
wrf_t2 = wrf_init['t2']
wrf_slp = wrf_init['slp']

In [7]:
print(wrf_lon.shape, wrf_lat.shape, wrf_time.shape, wrf_velocity.shape, wrf_degree.shape)

(80,) (128,) (12192,) (12192, 128, 80) (12192, 128, 80)


## 构造real数据点

In [8]:
# 分组提取数据
df_group = real_target.groupby('站位编号')

# time
real_time_00 = df_group.get_group(0)['获取时间'].values
real_time_01 = df_group.get_group(1)['获取时间'].values
real_time_02 = df_group.get_group(2)['获取时间'].values
real_time_03 = df_group.get_group(3)['获取时间'].values

# lat
real_lat_00 = df_group.mean(numeric_only=True)['纬度'].loc[0]
real_lat_01 = df_group.mean(numeric_only=True)['纬度'].loc[1]
real_lat_02 = df_group.mean(numeric_only=True)['纬度'].loc[2]
real_lat_03 = df_group.mean(numeric_only=True)['纬度'].loc[3]

# lon
real_lon_00 = df_group.mean(numeric_only=True)['经度'].loc[0]
real_lon_01 = df_group.mean(numeric_only=True)['经度'].loc[1]
real_lon_02 = df_group.mean(numeric_only=True)['经度'].loc[2]
real_lon_03 = df_group.mean(numeric_only=True)['经度'].loc[3]

# velocity_10
real_velocity_00 = df_group.get_group(0)['平均风速_10m'].values
real_velocity_01 = df_group.get_group(1)['平均风速_10m'].values
real_velocity_02 = df_group.get_group(2)['平均风速_10m'].values
real_velocity_03 = df_group.get_group(3)['平均风速_10m'].values

# degree
real_degree_00 = df_group.get_group(0)['平均风向'].values
real_degree_01 = df_group.get_group(1)['平均风向'].values
real_degree_02 = df_group.get_group(2)['平均风向'].values
real_degree_03 = df_group.get_group(3)['平均风向'].values

In [9]:
print(real_lat_00, real_lon_00)
print(real_lat_01, real_lon_01)
print(real_lat_02, real_lon_02)
print(real_lat_03, real_lon_03)

21.208984664802514 113.72401698357217
21.353082562039315 113.96000312714988
21.562553006905812 113.45516699995203
20.891871804646385 113.78421043255038


## 反距离权重插值

In [10]:
## IDW实现函数
# 计算两点欧式距离
def haversine(lon1, lat1, lon2, lat2):
    R = 6372.8
    dLon = radians(lon2 - lon1)
    dLat = radians(lat2 - lat1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2  #半正矢公式
    c = 2 * asin(sqrt(a)) #求解两点间中心角，大圆弧对应的球面角
    d = R*c
    return d

# 反距离加权计算
def IDW(y, x, z, yi, xi):
    lstxyzi = []
    for p in range(len(xi)):
        lstdist = []
        for s in range(len(x)):
            d = (haversine(x[s], y[s], xi[p], yi[p]))
            lstdist.append(d)
        sumsup = list((1 / np.power(lstdist, 2)))
        suminf = np.sum(sumsup)
        sumsup = np.sum(np.array(sumsup) * np.array(z))
        u = sumsup / suminf
        xyzi = [xi[p] , yi[p] , u]
        lstxyzi.append(xyzi)
    return(lstxyzi)

In [11]:
# 构建网格数据的所有空间点
lat_grid, lon_grid = np.meshgrid(wrf_lat, wrf_lon, indexing='ij')
position_origin = np.column_stack((lat_grid.flatten(), lon_grid.flatten()))
x_origin = position_origin[:, 0]
y_origin = position_origin[:, 1]

In [12]:
lat_grid.shape,lon_grid.shape

((128, 80), (128, 80))

In [13]:
len(x_origin)

10240

In [14]:
position_origin.shape

(10240, 2)

In [17]:
## 00
distance_00 = []
for step, i in enumerate(range(len(x_origin))):
    distance_00.append(haversine(real_lon_00, real_lat_00, y_origin[i], x_origin[i]))

distance_00 = np.array(distance_00)
index_position_00 = np.where(distance_00 < 50)

x_origin_target_00 = x_origin[index_position_00]
y_origin_target_00 = y_origin[index_position_00]

In [18]:
## 01
distance_01 = []
for step, i in enumerate(range(len(x_origin))):
    distance_01.append(haversine(real_lon_01, real_lat_01, y_origin[i], x_origin[i]))

distance_01 = np.array(distance_01)
index_position_01 = np.where(distance_01 < 50)
# print(index_position_01)

x_origin_target_01 = x_origin[index_position_01]
y_origin_target_01 = y_origin[index_position_01]

In [19]:
## 02
distance_02 = []
for step, i in enumerate(range(len(x_origin))):
    distance_02.append(haversine(real_lon_02, real_lat_02, y_origin[i], x_origin[i]))

distance_02 = np.array(distance_02)
index_position_02 = np.where(distance_02 < 50)
# print(index_position_02)

x_origin_target_02 = x_origin[index_position_02]
y_origin_target_02 = y_origin[index_position_02]

In [20]:
## 03
distance_03 = []
for step, i in enumerate(range(len(x_origin))):
    distance_03.append(haversine(real_lon_03, real_lat_03, y_origin[i], x_origin[i]))

distance_03 = np.array(distance_03)
index_position_03 = np.where(distance_03 < 50)
# print(index_position_03)

x_origin_target_03 = x_origin[index_position_03]
y_origin_target_03 = y_origin[index_position_03]

In [21]:
# 目标插值点--00
x_target_00 = np.array([real_lat_00])
y_target_00 = np.array([real_lon_00])

# 目标插值点--01
x_target_01 = np.array([real_lat_01])
y_target_01 = np.array([real_lon_01])

# 目标插值点--02
x_target_02 = np.array([real_lat_02])
y_target_02 = np.array([real_lon_02])

# 目标插值点--03
x_target_03 = np.array([real_lat_03])
y_target_03 = np.array([real_lon_03])

In [22]:
np.size(wrf_time)

12192

## 创建存储容器

In [23]:
# 00
velocity10_target_00 = np.empty((np.size(wrf_time), np.size(x_target_00)))
degree_target_00 = np.empty((np.size(wrf_time), np.size(x_target_00)))
u10_target_00 = np.empty((np.size(wrf_time), np.size(x_target_00)))
v10_target_00 = np.empty((np.size(wrf_time), np.size(x_target_00)))
rh2_target_00 = np.empty((np.size(wrf_time), np.size(x_target_00)))
t2_target_00 = np.empty((np.size(wrf_time), np.size(x_target_00)))
slp_target_00 = np.empty((np.size(wrf_time), np.size(x_target_00)))

In [24]:
# 01
velocity10_target_01 = np.empty((np.size(wrf_time), np.size(x_target_01)))
degree_target_01 = np.empty((np.size(wrf_time), np.size(x_target_01)))
u10_target_01 = np.empty((np.size(wrf_time), np.size(x_target_01)))
v10_target_01 = np.empty((np.size(wrf_time), np.size(x_target_01)))
rh2_target_01 = np.empty((np.size(wrf_time), np.size(x_target_01)))
t2_target_01 = np.empty((np.size(wrf_time), np.size(x_target_01)))
slp_target_01 = np.empty((np.size(wrf_time), np.size(x_target_01)))

In [25]:
# 02
velocity10_target_02 = np.empty((np.size(wrf_time), np.size(x_target_02)))
degree_target_02 = np.empty((np.size(wrf_time), np.size(x_target_02)))
u10_target_02 = np.empty((np.size(wrf_time), np.size(x_target_02)))
v10_target_02 = np.empty((np.size(wrf_time), np.size(x_target_02)))
rh2_target_02 = np.empty((np.size(wrf_time), np.size(x_target_02)))
t2_target_02 = np.empty((np.size(wrf_time), np.size(x_target_02)))
slp_target_02 = np.empty((np.size(wrf_time), np.size(x_target_02)))

In [26]:
# 03
velocity10_target_03 = np.empty((np.size(wrf_time), np.size(x_target_03)))
degree_target_03 = np.empty((np.size(wrf_time), np.size(x_target_03)))
u10_target_03 = np.empty((np.size(wrf_time), np.size(x_target_03)))
v10_target_03 = np.empty((np.size(wrf_time), np.size(x_target_03)))
rh2_target_03 = np.empty((np.size(wrf_time), np.size(x_target_03)))
t2_target_03 = np.empty((np.size(wrf_time), np.size(x_target_03)))
slp_target_03 = np.empty((np.size(wrf_time), np.size(x_target_03)))

In [27]:
x_target_00

array([21.20898466])

In [28]:
# 循环计算每个时间步 反距离插值--00
for i, t in enumerate(wrf_time):
    # 每个时间步的原始信息场
    velocity10_current = wrf_velocity[i, :].flatten()
    degree_current = wrf_degree[i, :].flatten()
    u10_current = wrf_u10[i, :].flatten()
    v10_current = wrf_v10[i, :].flatten()
    rh2_current = wrf_rh2[i, :].flatten()
    t2_current = wrf_t2[i, :].flatten()
    slp_current = wrf_slp[i, :].flatten()
    # 反距离插值
    velocity10_target_00[i, :] = np.array(IDW(x_origin_target_00, y_origin_target_00, velocity10_current[index_position_00], x_target_00, y_target_00))[:, -1]
    degree_target_00[i, :] = np.array(IDW(x_origin_target_00, y_origin_target_00, degree_current[index_position_00], x_target_00, y_target_00))[:, -1]
    u10_target_00[i, :] = np.array(IDW(x_origin_target_00, y_origin_target_00, u10_current[index_position_00], x_target_00, y_target_00))[:, -1]
    v10_target_00[i, :] = np.array(IDW(x_origin_target_00, y_origin_target_00, v10_current[index_position_00], x_target_00, y_target_00))[:, -1]
    rh2_target_00[i, :] = np.array(IDW(x_origin_target_00, y_origin_target_00, rh2_current[index_position_00], x_target_00, y_target_00))[:, -1]
    t2_target_00[i, :] = np.array(IDW(x_origin_target_00, y_origin_target_00, t2_current[index_position_00], x_target_00, y_target_00))[:, -1]
    slp_target_00[i, :] = np.array(IDW(x_origin_target_00, y_origin_target_00, slp_current[index_position_00], x_target_00, y_target_00))[:, -1]

In [29]:
x_origin_target_00.shape, y_origin_target_00.shape

((72,), (72,))

In [30]:
velocity10_current[index_position_00].shape

(72,)

In [31]:
x_target_00

array([21.20898466])

In [32]:
velocity10_target_00.shape

(12192, 1)

In [33]:
# 循环计算每个时间步 反距离插值--01
for i, t in enumerate(wrf_time):
    # 每个时间步的原始信息场
    velocity10_current = wrf_velocity[i, :].flatten()
    degree_current = wrf_degree[i, :].flatten()
    u10_current = wrf_u10[i, :].flatten()
    v10_current = wrf_v10[i, :].flatten()
    rh2_current = wrf_rh2[i, :].flatten()
    t2_current = wrf_t2[i, :].flatten()
    slp_current = wrf_slp[i, :].flatten()
    # 反距离插值
    velocity10_target_01[i, :] = np.array(IDW(x_origin_target_01, y_origin_target_01, velocity10_current[index_position_01], x_target_01, y_target_01))[:, -1]
    degree_target_01[i, :] = np.array(IDW(x_origin_target_01, y_origin_target_01, degree_current[index_position_01], x_target_01, y_target_01))[:, -1]
    u10_target_01[i, :] = np.array(IDW(x_origin_target_01, y_origin_target_01, u10_current[index_position_01], x_target_01, y_target_01))[:, -1]
    v10_target_01[i, :] = np.array(IDW(x_origin_target_01, y_origin_target_01, v10_current[index_position_01], x_target_01, y_target_01))[:, -1]
    rh2_target_01[i, :] = np.array(IDW(x_origin_target_01, y_origin_target_01, rh2_current[index_position_01], x_target_01, y_target_01))[:, -1]
    t2_target_01[i, :] = np.array(IDW(x_origin_target_01, y_origin_target_01, t2_current[index_position_01], x_target_01, y_target_01))[:, -1]
    slp_target_01[i, :] = np.array(IDW(x_origin_target_01, y_origin_target_01, slp_current[index_position_01], x_target_01, y_target_01))[:, -1]

In [34]:
# 循环计算每个时间步 反距离插值--02
for i, t in enumerate(wrf_time):
    # 每个时间步的原始信息场
    velocity10_current = wrf_velocity[i, :].flatten()
    degree_current = wrf_degree[i, :].flatten()
    u10_current = wrf_u10[i, :].flatten()
    v10_current = wrf_v10[i, :].flatten()
    rh2_current = wrf_rh2[i, :].flatten()
    t2_current = wrf_t2[i, :].flatten()
    slp_current = wrf_slp[i, :].flatten()
    # 反距离插值
    velocity10_target_02[i, :] = np.array(IDW(x_origin_target_02, y_origin_target_02, velocity10_current[index_position_02], x_target_02, y_target_02))[:, -1]
    degree_target_02[i, :] = np.array(IDW(x_origin_target_02, y_origin_target_02, degree_current[index_position_02], x_target_02, y_target_02))[:, -1]
    u10_target_02[i, :] = np.array(IDW(x_origin_target_02, y_origin_target_02, u10_current[index_position_02], x_target_02, y_target_02))[:, -1]
    v10_target_02[i, :] = np.array(IDW(x_origin_target_02, y_origin_target_02, v10_current[index_position_02], x_target_02, y_target_02))[:, -1]
    rh2_target_02[i, :] = np.array(IDW(x_origin_target_02, y_origin_target_02, rh2_current[index_position_02], x_target_02, y_target_02))[:, -1]
    t2_target_02[i, :] = np.array(IDW(x_origin_target_02, y_origin_target_02, t2_current[index_position_02], x_target_02, y_target_02))[:, -1]
    slp_target_02[i, :] = np.array(IDW(x_origin_target_02, y_origin_target_02, slp_current[index_position_02], x_target_02, y_target_02))[:, -1]

In [35]:
# 循环计算每个时间步 反距离插值--03
for i, t in enumerate(wrf_time):
    # 每个时间步的原始信息场
    velocity10_current = wrf_velocity[i, :].flatten()
    degree_current = wrf_degree[i, :].flatten()
    u10_current = wrf_u10[i, :].flatten()
    v10_current = wrf_v10[i, :].flatten()
    rh2_current = wrf_rh2[i, :].flatten()
    t2_current = wrf_t2[i, :].flatten()
    slp_current = wrf_slp[i, :].flatten()
    # 反距离插值
    velocity10_target_03[i, :] = np.array(IDW(x_origin_target_03, y_origin_target_03, velocity10_current[index_position_03], x_target_03, y_target_03))[:, -1]
    degree_target_03[i, :] = np.array(IDW(x_origin_target_03, y_origin_target_03, degree_current[index_position_03], x_target_03, y_target_03))[:, -1]
    u10_target_03[i, :] = np.array(IDW(x_origin_target_03, y_origin_target_03, u10_current[index_position_03], x_target_03, y_target_03))[:, -1]
    v10_target_03[i, :] = np.array(IDW(x_origin_target_03, y_origin_target_03, v10_current[index_position_03], x_target_03, y_target_03))[:, -1]
    rh2_target_03[i, :] = np.array(IDW(x_origin_target_03, y_origin_target_03, rh2_current[index_position_03], x_target_03, y_target_03))[:, -1]
    t2_target_03[i, :] = np.array(IDW(x_origin_target_03, y_origin_target_03, t2_current[index_position_03], x_target_03, y_target_03))[:, -1]
    slp_target_03[i, :] = np.array(IDW(x_origin_target_03, y_origin_target_03, slp_current[index_position_03], x_target_03, y_target_03))[:, -1]

In [36]:
velocity10_target_00.shape

(12192, 1)

In [37]:
# 保存数据--00
sio.savemat(os.path.join('../data', 'idw_00_100.mat'), 
            {'idw_velocity10': velocity10_target_00, 'idw_degree': degree_target_00,
             'idw_u10': u10_target_00, 'idw_v10': v10_target_00, 
             'idw_rh2': rh2_target_00, 'idw_t2': t2_target_00,
             'idw_slp': slp_target_00})

In [38]:
# 保存数据--01
sio.savemat(os.path.join('../data', 'idw_01_100.mat'), 
            {'idw_velocity10': velocity10_target_01, 'idw_degree': degree_target_01,
             'idw_u10': u10_target_01, 'idw_v10': v10_target_01, 
             'idw_rh2': rh2_target_01, 'idw_t2': t2_target_01,
             'idw_slp': slp_target_01})

In [39]:
# 保存数据--02
sio.savemat(os.path.join('../data', 'idw_02_100.mat'), 
            {'idw_velocity10': velocity10_target_02, 'idw_degree': degree_target_02,
             'idw_u10': u10_target_02, 'idw_v10': v10_target_02, 
             'idw_rh2': rh2_target_02, 'idw_t2': t2_target_02,
             'idw_slp': slp_target_02})

In [40]:
# 保存数据--03
sio.savemat(os.path.join('../data', 'idw_03_100.mat'), 
            {'idw_velocity10': velocity10_target_03, 'idw_degree': degree_target_03,
             'idw_u10': u10_target_03, 'idw_v10': v10_target_03, 
             'idw_rh2': rh2_target_03, 'idw_t2': t2_target_03,
             'idw_slp': slp_target_03})